In [9]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms, models
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
# Load an image for prediction (for example, from a file)
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.models import load_model
from tensorflow.keras.utils import load_img, img_to_array
import cv2

import numpy as np

import albumentations as A
from albumentations.pytorch import ToTensorV2

import os
import cv2
import torch
import timm
import time
import numpy as np
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2


In [2]:
import os
import shutil
from sklearn.model_selection import train_test_split

input_dir = "../Datasets/LFR Dataset/clean"
output_dir = "../Datasets/LFR Dataset/split"
split_ratios = [0.7, 0.15, 0.15]  # train, val, test

for class_name in os.listdir(input_dir):
    class_path = os.path.join(input_dir, class_name)
    if not os.path.isdir(class_path):
        continue

    images = [f for f in os.listdir(class_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    train, temp = train_test_split(images, train_size=split_ratios[0], random_state=42)
    val, test = train_test_split(temp, test_size=0.5, random_state=42)

    for split_name, split_files in zip(["train", "val", "test"], [train, val, test]):
        split_class_dir = os.path.join(output_dir, split_name, class_name)
        os.makedirs(split_class_dir, exist_ok=True)
        for img in split_files:
            src_path = os.path.join(class_path, img)
            dst_path = os.path.join(split_class_dir, img)
            shutil.copyfile(src_path, dst_path)

print("Splitting complete.")


KeyboardInterrupt: 

In [10]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

def get_transforms(split="train"):
    if split == "train":
        return A.Compose([
            A.Resize(128, 128),
            A.HorizontalFlip(p=0.5),
            A.RandomBrightnessContrast(p=0.5),
            A.GaussNoise(p=0.3),
            A.Blur(blur_limit=3, p=0.2),
            A.Rotate(limit=15, p=0.5),
            A.RandomGamma(p=0.3),
            A.GridDistortion(p=0.2),
            A.CoarseDropout(max_holes=8, max_height=8, max_width=8, p=0.5),
            A.CLAHE(p=0.3),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
            ToTensorV2()
        ])
    else:
        return A.Compose([
            A.Resize(128, 128),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
            ToTensorV2()
        ])


In [12]:
from torch.utils.data import Dataset
import os
import cv2

class LFRDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = sorted(os.listdir(root_dir))
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.classes)}
        self.samples = []

        for cls in self.classes:
            cls_path = os.path.join(root_dir, cls)
            for img_name in os.listdir(cls_path):
                if img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                    self.samples.append((os.path.join(cls_path, img_name), self.class_to_idx[cls]))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transform:
            image = self.transform(image=image)['image']
        return image, label


In [13]:


from torch.utils.data import DataLoader
import os

# Adjust path to your dataset location in Google Drive
base_path = "../Datasets/LFR Dataset/split"


# Create datasets
train_dataset = LFRDataset(os.path.join(base_path, "train"), transform=get_transforms("train"))
val_dataset   = LFRDataset(os.path.join(base_path, "val"), transform=get_transforms("val"))
test_dataset  = LFRDataset(os.path.join(base_path, "test"), transform=get_transforms("val"))

# Create data loaders (keep num_workers low in Colab)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=2)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

# Print dataset sizes
print(f"Train: {len(train_dataset)}, Val: {len(val_dataset)}, Test: {len(test_dataset)}")


C:\Users\Anas\AppData\Local\Temp\ipykernel_5276\2507135441.py:15: UserWarning: Argument(s) 'max_holes, max_height, max_width' are not valid for transform CoarseDropout
  A.CoarseDropout(max_holes=8, max_height=8, max_width=8, p=0.5),


Train: 98506, Val: 21108, Test: 21111


In [15]:
# Install timm if not already installed
!pip install timm -q

import torch
import torch.nn as nn
import timm

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Load pretrained EfficientNet-B0 from TIMM
model = timm.create_model('efficientnet_b0', pretrained=True)

# Freeze the base model (optional)
for param in model.parameters():
    param.requires_grad = False  # You can unfreeze later for fine-tuning

# Replace the classifier head for 3-class output (front, left, right)
model.classifier = nn.Sequential(
    nn.Dropout(0.3),
    nn.Linear(model.classifier.in_features, 3)  # 3 output classes
)

# Send model to GPU/CPU
model = model.to(device)



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Using device: cpu


In [16]:
class_labels = {"front": 0, "left": 1, "right": 2}
idx_to_class = {v: k for k, v in class_labels.items()}


In [17]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=1e-4, weight_decay=1e-5)


In [18]:
def evaluate_model(model, dataloader, device=device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, dim=1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    accuracy = correct / total
    print(f"Accuracy: {accuracy * 100:.2f}%")
    return accuracy


In [19]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10, device=device):
    best_val_acc = 0.0

    for epoch in range(num_epochs):
        print(f"\nEpoch [{epoch+1}/{num_epochs}]")
        start_time = time.time()

        model.train()
        train_loss, train_correct = 0.0, 0
        train_bar = tqdm(train_loader, desc="Training", leave=False)

        for images, labels in train_bar:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            train_correct += (preds == labels).sum().item()
            train_bar.set_postfix(loss=loss.item())

        train_acc = train_correct / len(train_loader.dataset)
        avg_train_loss = train_loss / len(train_loader.dataset)

        # Validation
        model.eval()
        val_loss, val_correct = 0.0, 0
        with torch.no_grad():
            val_bar = tqdm(val_loader, desc="Validating", leave=False)
            for images, labels in val_bar:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * images.size(0)
                _, preds = torch.max(outputs, 1)
                val_correct += (preds == labels).sum().item()
                val_bar.set_postfix(loss=loss.item())

        val_acc = val_correct / len(val_loader.dataset)
        avg_val_loss = val_loss / len(val_loader.dataset)

        print(f"Epoch {epoch+1} done in {time.time() - start_time:.1f}s — "
              f"Train Loss: {avg_train_loss:.4f}, Acc: {train_acc:.4f} | "
              f"Val Loss: {avg_val_loss:.4f}, Acc: {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), "best_model.pth")
            print("✅ New best model saved.")

    print("Training complete.")


In [ ]:
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=15)



Epoch [1/15]


Training:   0%|          | 0/1540 [00:00<?, ?it/s]

In [1]:
# Save model weights only
torch.save(model.state_dict(), "efficientnet_face_direction.pth")
print("Model saved successfully.")


NameError: name 'torch' is not defined